In [1]:
!pip install thulac

In [2]:
!pip install pytorch_transformers

In [5]:
import torch, gc, json, os, thulac, string, re, requests, hashlib, urllib.parse 
import numpy as np
from datetime import datetime
from pytorch_transformers import *

from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=6, random_state=0, init='k-means++', max_iter=10)

In [9]:
def md5(str):
    m = hashlib.md5()
    m.update(str.encode("utf8"))
    return m.hexdigest()

BASE_DIR = './drive/MyDrive/NLP/WantWords/'

device = torch.device('cpu')
torch.backends.cudnn.benchmark = True
words_t = torch.tensor(np.array([0]))
itemsPerCol = 20
GET_NUM = 100
NUM_RESPONSE = 500
words_t = torch.tensor(np.array([0]))

tokenizer_class = BertTokenizer
tokenizer_Ch = tokenizer_class.from_pretrained('bert-base-chinese')
tokenizer_En = tokenizer_class.from_pretrained('bert-base-uncased')
#========================ChineseRD
MODE = 'Psc'
lac = thulac.thulac()

def load_data():
    (word2index, index2word, _, _, _, _, _) = np.load(BASE_DIR + 'data_features/data_inUse1.npy', allow_pickle=True)
    wd_charas = np.load(BASE_DIR + 'data_features/data_inUse2.npy', allow_pickle=True)
    ((_, _, _, wd_sems, wd_POSs),(_, mask_s)) = np.load(BASE_DIR + 'data_features/data_inUse3.npy', allow_pickle=True)
    mask_s = torch.from_numpy(mask_s).to(device)
    wd_POSs = torch.from_numpy(wd_POSs).float().to(device)
    wd_charas = torch.from_numpy(wd_charas).float().to(device)
    wd_sems = torch.from_numpy(wd_sems).float().to(device)
    wd_C = []
    mask_c = []
    mask_s = mask_s.float()
    return word2index, index2word, (wd_C, wd_sems, wd_POSs, wd_charas), (mask_c, mask_s)

word2index, index2word, wd_features, mask_ = load_data()
(wd_C, wd_sems, wd_POSs, wd_charas) = wd_features
(mask_c, mask_s) = mask_
index2word = np.array(index2word)

# 添加同义词词林用于描述为一个词时的同义词推荐
index2synset = [[] for i in range(len(word2index))]
for line in open(BASE_DIR + 'data_definition_synset/word2synset_synset.txt').readlines():
    wd = line.split()[0]
    synset = line.split()[1:]
    for syn in synset:
        index2synset[word2index[wd]].append(word2index[syn])

MODEL_FILE = BASE_DIR + 'models/Zh.model'
model = torch.load(MODEL_FILE, map_location=lambda storage, loc: storage)
model.eval()
wd_data_ = json.load(open(BASE_DIR+'data_definition_synset/wd_def_for_website_zh.json'))

#wd_data = dict()
wd_data = wd_data_.copy()
wd_defi = wd_data_.copy()
for wd in wd_data_:
    #wd_data[wd] = {'w': wd_data_[wd]['word'], 'd': wd_data_[wd]['definition'], 'P': wd_data_[wd]['POS'], 'l': wd_data_[wd]['length'], 'b': wd_data_[wd]['bihuashu'], 'B': wd_data_[wd]['bihuashu1st'], 'p': wd_data_[wd]['pinyin'], 's': wd_data_[wd]['pinyinshouzimu'], 'r': wd_data_[wd]['rhyme']}
    wd_data[wd] = {'w': wd_data_[wd]['word'], 'P': wd_data_[wd]['POS'], 'l': wd_data_[wd]['length'], 'b': wd_data_[wd]['bihuashu'], 'B': wd_data_[wd]['bihuashu1st'], 'p': wd_data_[wd]['pinyin'], 's': wd_data_[wd]['pinyinshouzimu'], 'r': wd_data_[wd]['rhyme']}
    wd_defi[wd] = wd_data_[wd]['definition']
del wd_data_


Model loaded succeed


/usr/local/lib/python3.7/dist-packages/torch/serialization.py:786: SourceChangeWarning: source code of class 'model.Encoder' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.7/dist-packages/torch/serialization.py:786: SourceChangeWarning: source code of class 'torch.nn.modules.sparse.Embedding' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.7/dist-packages/torch/serialization.py:786: SourceChangeWarning: source code of class 'torch.nn.modules.dropout.Dropout' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.du

In [20]:
def Score2Hexstr(score, maxsc):
    thr = maxsc/1.5
    l = len(score)
    ret = ['00']*l
    for i in range(l):
        res = int(200*(score[i] - thr)/thr)
        if res>15:
            ret[i] = hex(res)[2:]
        else:
            break
    return 

# description = request.GET['description']
# RD_mode = request.GET['mode']
description = '邓小平'
RD_mode = 'CC'
with torch.no_grad():
    def_words = [w for w, p in lac.cut(description)]
    def_word_idx = []
    if len(def_words) > 0:
        for def_word in def_words:
            if def_word in word2index:
                def_word_idx.append(word2index[def_word])
            else:
                for dw in def_word:
                    try:
                        def_word_idx.append(word2index[dw])
                    except:
                        def_word_idx.append(word2index['<OOV>'])
        x_len = len(def_word_idx)
        if set(def_word_idx)=={word2index['<OOV>']}:
            x_len = 1
        if x_len==1:
            if def_word_idx[0]>1:
                score = ((model.embedding.weight.data).mm((model.embedding.weight.data[def_word_idx[0]]).unsqueeze(1))).squeeze(1)
                if RD_mode=='CC':
                    score[def_word_idx[0]] = -10.
                score[np.array(index2synset[def_word_idx[0]])] *= 2
                sc, indices = torch.sort(score, descending=True)
                predicted = indices[:NUM_RESPONSE].detach().cpu().numpy()
                score = sc[:NUM_RESPONSE].detach().numpy()
                maxsc = sc[0].detach().item()
                s2h = Score2Hexstr(score, maxsc)
            else:
                predicted= []
                ret = {'error': 1} # 字符无法识别
        else:
            defi = '[CLS] ' + description
            def_word_idx = tokenizer_Ch.encode(defi)[:80]
            def_word_idx.extend(tokenizer_Ch.encode('[SEP]'))
            definition_words_t = torch.tensor(np.array(def_word_idx), dtype=torch.int64, device=device)
            definition_words_t = definition_words_t.unsqueeze(0) # batch_size = 1
            score = model('test', x=definition_words_t, w=words_t, ws=wd_sems, wP=wd_POSs, wc=wd_charas, wC=wd_C, msk_s=mask_s, msk_c=mask_c, mode=MODE)
            sc, indices = torch.sort(score, descending=True)
            predicted = indices[0, :NUM_RESPONSE].detach().cpu().numpy()
            score = sc[0, :NUM_RESPONSE].detach().numpy()
            maxsc = sc[0, 0].detach().item()
            s2h = Score2Hexstr(score, maxsc)
    else:
        predicted= []
        ret = {'error': 0} # 输入为空
if len(predicted)>0:
    res = index2word[predicted]
    ret = [] 
    cn = -1
    if RD_mode=='CC':
        def_words = set(def_words)
        for wd in res:
            cn += 1
            if wd not in def_words:
                try:
                    ret.append(wd_data[wd])
                    ret[len(ret)-1]['c'] = s2h[cn]
                except:
                    continue
    else:
        for wd in res:
            cn += 1
            try:
                ret.append(wd_data[wd])
                ret[len(ret)-1]['c'] = s2h[cn]
            except:
                continue
# with open('ret.txt', 'w', encoding='utf-8') as f:
#     f.write(str(ret))

for i in ret[:10]:
    print(i['w'])
    print('\n')     

社会主义革命


新民主主义革命


南昌起义


马克思列宁主义


中共中央


毛泽东思想


中国共产党


中国工农红军


共产国际


苏区




In [16]:
for i in ret[:10]:
    print(i['w'])
    print('\n') 

杜甫


杜牧


诗仙


陶渊明


白居易


诗圣


李商隐


孟浩然


柳宗元


屈原




In [ ]:
def getClass2Class(r, score): 
    perCluster = [[],[],[],[],[],[]]
    for i in range(GET_NUM):
        perCluster[r[i]].append(score[i])
    scorePC = []
    for i in range(6):
        l = len(perCluster[i]) if len(perCluster[i])<5 else 5
        scorePC.append(sum(perCluster[i][:l])/l)
    ind = [indsc[0] for indsc in sorted(enumerate(scorePC), key=lambda x:x[1], reverse=True)]
    class2class = [0,0,0,0,0,0]
    for i in range(6):
        class2class[ind[i]] = i
    return class2class 

In [ ]:
def ChineseRDCluster(request):
    description = request.GET['description']
    RD_mode = request.GET['mode']
    with torch.no_grad():
        def_words = [w for w, p in lac.cut(description)]
        def_word_idx = []
        if len(def_words) > 0:
            for def_word in def_words:
                if def_word in word2index:
                    def_word_idx.append(word2index[def_word])
                else:
                    for dw in def_word:
                        try:
                            def_word_idx.append(word2index[dw])
                        except:
                            def_word_idx.append(word2index['<OOV>'])
            x_len = len(def_word_idx)
            if set(def_word_idx)=={word2index['<OOV>']}:
                x_len = 1
            if x_len==1:
                if def_word_idx[0]>1:
                    score = ((model.embedding.weight.data).mm((model.embedding.weight.data[def_word_idx[0]]).unsqueeze(1))).squeeze(1)
                    if RD_mode=='CC':
                        score[def_word_idx[0]] = -10.
                    score[np.array(index2synset[def_word_idx[0]])] *= 2
                    sc, indices = torch.sort(score, descending=True)
                    predicted = indices[:GET_NUM].detach().cpu().numpy()
                    score = sc[:GET_NUM].detach().numpy()
                    maxsc = sc[0].detach().item()
                    s2h = Score2Hexstr(score, maxsc)
                    r = kmeans.fit_predict(model.embedding.weight.data[predicted[:GET_NUM]].cpu().numpy()) # GET_NUM
                    class2class = getClass2Class(r, score[:GET_NUM])
                else:
                    predicted= []
                    ret = {'error': 1} # 字符无法识别
            else:
                defi = '[CLS] ' + description
                def_word_idx = tokenizer_Ch.encode(defi)[:80]
                def_word_idx.extend(tokenizer_Ch.encode('[SEP]'))
                definition_words_t = torch.tensor(np.array(def_word_idx), dtype=torch.int64, device=device)
                definition_words_t = definition_words_t.unsqueeze(0) # batch_size = 1
                score = model('test', x=definition_words_t, w=words_t, ws=wd_sems, wP=wd_POSs, wc=wd_charas, wC=wd_C, msk_s=mask_s, msk_c=mask_c, mode=MODE)
                sc, indices = torch.sort(score, descending=True)
                predicted = indices[0, :GET_NUM].detach().cpu().numpy()
                score = sc[0, :GET_NUM].detach().numpy()
                maxsc = sc[0, 0].detach().item()
                s2h = Score2Hexstr(score, maxsc)
                r = kmeans.fit_predict(model.embedding.weight.data[predicted[:GET_NUM]].cpu().numpy()) # GET_NUM
                class2class = getClass2Class(r, score[:GET_NUM])
        else:
            predicted= []
            ret = {'error': 0} # 输入为空
    if len(predicted)>0:
        res = index2word[predicted]
        ret = [] 
        cn = -1
        if RD_mode=='CC':
            def_words = set(def_words)
            for wd in res:
                cn += 1
                if wd not in def_words:
                    try:
                        ret.append(wd_data[wd])
                        ret[len(ret)-1]['c'] = s2h[cn]
                        ret[len(ret)-1]['C'] = class2class[int(r[cn])] # 必须转为int，否则其实是int64类型，会报不能json序列化的错误
                        ret[len(ret)-1]['d'] = wd_defi[wd]
                        ret.sort(key=lambda x: x['C'])
                    except:
                        continue
        else:
            for wd in res:
                cn += 1
                try:
                    ret.append(wd_data[wd])
                    ret[len(ret)-1]['c'] = s2h[cn]
                    ret[len(ret)-1]['C'] = class2class[int(r[cn])] # 必须转为int，否则其实是int64类型，会报不能json序列化的错误
                    ret[len(ret)-1]['d'] = wd_defi[wd]
                    ret.sort(key=lambda x: x['C'])
                except:
                    continue
    return HttpResponse(json.dumps(ret,ensure_ascii=False),content_type="application/json,charset=utf-8")